### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


### Download Resources

Use the following syntax for downloading resources:

- **From civit.ai**: Use the command `!wget -c "link" -O "output_path/resource_name"`
  - **Example**: `!wget -c "https://civitai.com/api/download/models/125985?type=Model&format=SafeTensor" -O "./models/loras/Ghibli_v6.safetensors"`
    - This example saves the model file with the name `Ghibli_v6.safetensors` in the `./models/loras/` directory.

- **From huggingface.ai**: Use the command `!wget -c "link" -P "output_path"`
  - Replace `"link"` with the Hugging Face URL and `"output_path"` with the desired directory path.
  - **Example**: `!wget -c "https://huggingface.co/Lykon/dreamshaper-8-lcm/resolve/main/DreamShaper8_LCM.safetensors" -P "./models/checkpoints/"`
    - This example saves the file in the `./models/huggingface/` directory.

In [ ]:
# z_image_turbo
!wget -c 'https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors' -O './ComfyUI/models/text_encoders/qwen_3_4b.safetensors'
!wget -c 'https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors' -O './ComfyUI/models/vae/ae.safetensors'
!wget -c 'https://huggingface.co/Kijai/Z-Image_comfy_fp8_scaled/resolve/main/z-image-turbo_fp8_scaled_e4m3fn_KJ.safetensors' -O './ComfyUI/models/diffusion_models/z-image-turbo_fp8_scaled_e4m3fn_KJ.safetensors'


### Run ComfyUI

In [ ]:
# --- One-cell ComfyUI + ngrok (pyngrok) ---
# Put your token here. Get it at ngrok.com:
NGROK_TOKEN = "YOUR TOKEN HERE"
PORT = 8188

!pip -q install pyngrok

import os, threading, time, socket, subprocess
from pyngrok import ngrok, conf

# Configure ngrok auth
conf.get_default().auth_token = NGROK_TOKEN

def wait_for_port(host="127.0.0.1", port=PORT, timeout=300):
    start = time.time()
    while time.time() - start < timeout:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            if sock.connect_ex((host, port)) == 0:
                return True
        time.sleep(0.5)
    return False

def start_tunnel_when_ready():
    print("Starting ComfyUI... waiting for it to listen on port", PORT)
    if wait_for_port("127.0.0.1", PORT, timeout=600):
        print("\nComfyUI is up. Starting ngrok tunnel...\n")
        t = ngrok.connect(addr=PORT, proto="http")
        print("✅ This is the URL to access ComfyUI:", t.public_url)
    else:
        print("❌ Timed out waiting for ComfyUI to open port", PORT)

# Start watcher thread that creates the tunnel once ComfyUI is live
threading.Thread(target=start_tunnel_when_ready, daemon=True).start()

# Launch ComfyUI (listen externally so ngrok can reach it)
%cd /content/ComfyUI
!python main.py --listen 0.0.0.0 --port 8188 --dont-print-server


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')